In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

Gaussian Naive Bayes (GNB) is a **generative probabilistic model** for **continuous features**. Let the dataset be:


$$
\mathcal{D} = {(x_i, y_i)}_{i=1}^n, \quad x_i \in \mathbb{R}^m, \ y_i \in {1, \dots, K}
$$

Then:

1. **Naive Bayes assumption**: each feature is independent given the class:
$$
P(x \mid y=c) = \prod_{j=1}^m P(x_j \mid y=c)
$$

2. **Gaussian distribution**: each feature (x_j) is normally distributed conditioned on the class (y=c):
$$
x_j \mid y=c \sim \mathcal{N}(\mu_{jc}, \sigma^2_{jc})
$$

So the **conditional probability density function** is:

$$
P(x_j \mid y=c) = \frac{1}{\sqrt{2\pi \sigma^2_{jc}}} \exp\Bigg(-\frac{(x_j - \mu_{jc})^2}{2 \sigma^2_{jc}}\Bigg)
$$


In [ ]:
class GaussianNB(object):
    def fit(self, X, y):
        '''Parameter estimation for Gaussian NB'''

        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # Initialise mean, var, and prior for each class.
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):

            #Get examples with label y=c
            X_c = X[y == c]

            #Estimate mean from the training examples of class c.
            self._mean[idx, :] = X_c.mean(axis=0)

            #Estimate variance from the training examples of class c.
            self._var[idx, :] = X_c.var(axis=0)

            #Estimate priors.
            self._priors[idx] = X_c.shape[0] / float(n_samples)

        print("Mean: ", self._mean)
        print("Variance: ", self._var)
        print("Priors: ", self._priors)

### Parameter Estimation

For each class (c):

* **Mean**:

$$
\mu_{jc} = \frac{1}{N_c} \sum_{i:y_i=c} x_{ij}
$$

* **Variance**:

$$
\sigma^2_{jc} = \frac{1}{N_c} \sum_{i:y_i=c} (x_{ij} - \mu_{jc})^2
$$

* **Class prior**:

$$
\pi_c = P(y=c) = \frac{N_c}{n}
$$


In [ ]:
class GaussianNB(object):
    def fit(self, X, y):
        '''Parameter estimation for Gaussian NB'''

        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # Initialise mean, var, and prior for each class.
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):

            #Get examples with label y=c
            X_c = X[y == c]

            #Estimate mean from the training examples of class c.
            self._mean[idx, :] = X_c.mean(axis=0)

            #Estimate variance from the training examples of class c.
            self._var[idx, :] = X_c.var(axis=0)

            #Estimate priors.
            self._priors[idx] = X_c.shape[0] / float(n_samples)

        print("Mean: ", self._mean)
        print("Variance: ", self._var)
        print("Priors: ", self._priors)

    def _calc_pdf(self, class_idx, X):
        '''Calculates probability density for samples for class label class_idx'''

        mean = self._mean[class_idx]
        var = np.diag(self._var[class_idx])
        z = np.power(2 * np.pi, X.shape[0]/2) * np.power(np.linalg.det(var), 1/2)
        return (1/z) * np.exp(-(1/2)*(X - mean).T @ (np.linalg.inv(var)) @ (X - mean))


### Likelihood of a Sample

Assuming independence between features:

$$
P(x \mid y=c) = \prod_{j=1}^m P(x_j \mid y=c) = \prod_{j=1}^m \frac{1}{\sqrt{2\pi \sigma^2_{jc}}} \exp\Big(-\frac{(x_j - \mu_{jc})^2}{2 \sigma^2_{jc}}\Big)
$$

In log-space

$$
\log P(x \mid y=c) = \sum_{j=1}^m \Big [-\frac{1}{2}\log(2\pi \sigma^2_{jc}) - \frac{(x_j - \mu_{jc})^2}{2\sigma^2_{jc}} \Big]
$$


In [ ]:
class GaussianNB(object):
    def fit(self, X, y):
        '''Parameter estimation for Gaussian NB'''

        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # Initialise mean, var, and prior for each class.
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):

            #Get examples with label y=c
            X_c = X[y == c]

            #Estimate mean from the training examples of class c.
            self._mean[idx, :] = X_c.mean(axis=0)

            #Estimate variance from the training examples of class c.
            self._var[idx, :] = X_c.var(axis=0)

            #Estimate priors.
            self._priors[idx] = X_c.shape[0] / float(n_samples)

        print("Mean: ", self._mean)
        print("Variance: ", self._var)
        print("Priors: ", self._priors)

    def _calc_pdf(self, class_idx, X):
        '''Calculates probability density for samples for class label class_idx'''

        mean = self._mean[class_idx]
        var = np.diag(self._var[class_idx])
        z = np.power(2 * np.pi, X.shape[0]/2) * np.power(np.linalg.det(var), 1/2)
        return (1/z) * np.exp(-(1/2)*(X - mean).T @ (np.linalg.inv(var)) @ (X - mean))

    def _calc_prod_likelihood_prior(self, X):
        '''Calculates product of likelihood and priors.'''

        self._prod_likelihood_prior = np.zeros((X.shape[0], len(self._classes)), dtype=np.float64)

        for x_idx, x in enumerate(X):
            for idx, c in enumerate(self._classes):
                self._prod_likelihood_prior[x_idx, c] = (np.log(self._calc_pdf(idx, x)) + np.log(self._priors[idx]))


### Posterior Probability

By Bayes’ theorem:

$$
P(y=c \mid x) = \frac{P(x \mid y=c) \pi_c}{\sum_{c'=1}^K P(x \mid y=c') \pi_{c'}}
$$

* `_calc_prod_likelihood_prior(X)` computes (\log P(x \mid y=c) + \log \pi_c) for each class.
* `predict_proba(X)` converts log-likelihoods to probabilities:

In [ ]:
class GaussianNB(object):
    def fit(self, X, y):
        '''Parameter estimation for Gaussian NB'''

        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # Initialise mean, var, and prior for each class.
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):

            #Get examples with label y=c
            X_c = X[y == c]

            #Estimate mean from the training examples of class c.
            self._mean[idx, :] = X_c.mean(axis=0)

            #Estimate variance from the training examples of class c.
            self._var[idx, :] = X_c.var(axis=0)

            #Estimate priors.
            self._priors[idx] = X_c.shape[0] / float(n_samples)

        print("Mean: ", self._mean)
        print("Variance: ", self._var)
        print("Priors: ", self._priors)

    def _calc_pdf(self, class_idx, X):
        '''Calculates probability density for samples for class label class_idx'''

        mean = self._mean[class_idx]
        var = np.diag(self._var[class_idx])
        z = np.power(2 * np.pi, X.shape[0]/2) * np.power(np.linalg.det(var), 1/2)
        return (1/z) * np.exp(-(1/2)*(X - mean).T @ (np.linalg.inv(var)) @ (X - mean))

    def _calc_prod_likelihood_prior(self, X):
        '''Calculates product of likelihood and priors.'''

        self._prod_likelihood_prior = np.zeros((X.shape[0], len(self._classes)), dtype=np.float64)

        for x_idx, x in enumerate(X):
            for idx, c in enumerate(self._classes):
                self._prod_likelihood_prior[x_idx, c] = (np.log(self._calc_pdf(idx, x)) + np.log(self._priors[idx]))

    def predict(self, X):
        '''Predicts class labels for each example'''
        
        self._calc_prod_likelihood_prior(X)

        return np.argmax(self._prod_likelihood_prior, axis=1)


### Prediction

The predicted class is the one with the **highest posterior probability**:

$$
\hat{y} = \arg\max_c P(y=c \mid x)
$$

In [ ]:
class GaussianNB(object):
    def fit(self, X, y):
        '''Parameter estimation for Gaussian NB'''

        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # Initialise mean, var, and prior for each class.
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):

            #Get examples with label y=c
            X_c = X[y == c]

            #Estimate mean from the training examples of class c.
            self._mean[idx, :] = X_c.mean(axis=0)

            #Estimate variance from the training examples of class c.
            self._var[idx, :] = X_c.var(axis=0)

            #Estimate priors.
            self._priors[idx] = X_c.shape[0] / float(n_samples)

        print("Mean: ", self._mean)
        print("Variance: ", self._var)
        print("Priors: ", self._priors)

    def _calc_pdf(self, class_idx, X):
        '''Calculates probability density for samples for class label class_idx'''

        mean = self._mean[class_idx]
        var = np.diag(self._var[class_idx])
        z = np.power(2 * np.pi, X.shape[0]/2) * np.power(np.linalg.det(var), 1/2)
        return (1/z) * np.exp(-(1/2)*(X - mean).T @ (np.linalg.inv(var)) @ (X - mean))

    def _calc_prod_likelihood_prior(self, X):
        '''Calculates product of likelihood and priors.'''

        self._prod_likelihood_prior = np.zeros((X.shape[0], len(self._classes)), dtype=np.float64)

        for x_idx, x in enumerate(X):
            for idx, c in enumerate(self._classes):
                self._prod_likelihood_prior[x_idx, c] = (np.log(self._calc_pdf(idx, x)) + np.log(self._priors[idx]))

    def predict(self, X):
        '''Predicts class labels for each example'''
        
        self._calc_prod_likelihood_prior(X)

        return np.argmax(self._prod_likelihood_prior, axis=1)

    def predict_proba(self, X):
        '''Calculates probability of each example belonging to different classes.'''

        self._calc_prod_likelihood_prior(X)

        return np.exp(self._prod_likelihood_prior) / np.expand_dims(np.sum(np.exp(self._prod_likelihood_prior), axis = 1), axis = 1)

| Concept            | Formula                                                                                                    | Code                            |
| ------------------ | ---------------------------------------------------------------------------------------------------------- | ------------------------------- |
| Feature likelihood | $(P(x_j \mid y=c) = \frac{1}{\sqrt{2\pi \sigma^2_{jc}}} \exp(-\frac{(x_j-\mu_{jc})^2}{2\sigma^2_{jc}}))$     | `_calc_pdf()`                   |
| Class prior        | $(\pi_c = N_c/n)$                                                                                            | `self._priors[idx]`             |
| Log-likelihood     | $(\log P(x \mid y=c) = \sum_j [-\frac12 \log(2\pi\sigma^2_{jc}) - \frac{(x_j-\mu_{jc})^2}{2\sigma^2_{jc}}])$ | `_calc_prod_likelihood_prior()` |
| Posterior          | $(P(y=c \mid x) = \frac{P(x\mid y=c)\pi_c}{\sum_{c'} P(x \mid y=c')\pi_{c'}})$                               | `predict_proba()`               |
| Prediction         | $(\hat{y} = \arg\max_c P(y=c \mid x))$                                                                       | `predict()`                     |
